
# IRENE-2980

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
from time import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
from invisible_cities.database import load_db
from invisible_cities.core.system_of_units_c import SystemOfUnits
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.core.wfm_functions as wfm
import invisible_cities.core.tbl_functions as tbl
import invisible_cities.core.peak_functions_c as cpf
import invisible_cities.core.pmaps_functions as pf
import invisible_cities.core.sensor_functions as sf

In [ ]:
from invisible_cities.core.core_functions import define_window

In [ ]:
from invisible_cities.core.core_functions import lrange

In [ ]:
from collections import namedtuple

In [ ]:
from   invisible_cities.core.params import S12Params as S12P

In [ ]:
from   invisible_cities.core.params import SensorParams

In [ ]:
from invisible_cities.cities.irene import Irene, IRENE

## Preproc Steps

**Irene takes care of the following chores**

1. Deconvolute RWF
2. Compute calibrated sum of PMTs (including MAU threshold)
3. Zero Suppression (ZS) in PMTs
4. S1 search (analysis specific)
5. S2 search (analysis specific) and rebin
6. ZS in SiPMs
7. Select S2 in ZS SiPMs 

For fast pre-proc, all the above steps must be computed on the fly (e.g, minimal access to disk) and
using pre-compiled (cython) functions in calculation-intensive parts (loops)

## Files

In [ ]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/2980/run_2980.gdc1next.000.next1el_2980.root.h5')
PMP_file = os.path.join(os.environ['IC_DATA'],'LSC/PMP_2980/run_2983.000.PMP.h5')
CONF_file = os.path.join(os.environ['IC_CONFIG'], 'irene_Kr2983.conf')

In [ ]:
h5rwf = False

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

In [ ]:
h5rwf

In [ ]:
event_number = (100, 101, 102)

In [ ]:
for i, _ in enumerate(event_number):
    print(i)

### pmtrwf and sipmrwf vectors

To get vectors use **get_vectors(file)** in tbl_functions

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT(2983)
units = SystemOfUnits()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

### Instance

In [ ]:
s1par = S12P(tmin       = 10 * units.mus,
                 tmax   = 590 * units.mus,
                 stride = 4,
                 lmin   = 6,
                 lmax   = 16,
                 rebin  = False)

s2par = S12P(tmin       = 590 * units.mus,
                 tmax   = 620 * units.mus,
                 stride = 40,
                 lmin   = 100,
                 lmax   = 1000,
                 rebin  = True)

irene = Irene(run_number  = 2980,
                 files_in    = [RWF_file],
                 file_out    = PMP_file,
                 compression = 'ZLIB4',
                 nprint      = 1,
                 n_baseline  = 38000,
                 thr_trigger = 5.0 * units.adc,
                 n_MAU       = 100,
                 thr_MAU     = 3.0 * units.adc,
                 thr_csum_s1 = 0.2 * units.adc,
                 thr_csum_s2 = 1.0 * units.adc,
                 n_MAU_sipm  = 100,
                 thr_sipm    = 5.0 * units.pes,
                 s1_params   = s1par,
                 s2_params   = s2par,
                 thr_sipm_s2 = 30. * units.pes)

In [ ]:
def irene_run(irene, evt):
    # Check that PMAPS tables are written correctly
    
    with tb.open_file(irene.input_files[0], "r") as h5in:
        with tb.open_file(irene.output_file, "w",
            filters = tbl.filters(irene.compression)) as pmap_file:

            irene._set_pmap_store(pmap_file)
            irene.eventsInfo = h5in.root.Run.events
            pmtrwf  = h5in.root.RD. pmtrwf
            sipmrwf = h5in.root.RD.sipmrwf

            CWF = irene.deconv_pmt(pmtrwf[evt])
            csum, csum_mau = irene.calibrated_pmt_sum(CWF)
            s1_ene, s1_indx = irene.csum_zs(csum_mau,
                                             threshold = irene.thr_csum_s1)
            s2_ene, s2_indx = irene.csum_zs(csum,
                                             threshold = irene.thr_csum_s2)

            sipmzs = irene.calibrated_signal_sipm(sipmrwf[evt])
            S1, S2 = irene.find_S12(s1_ene, s1_indx, s2_ene, s2_indx)
            S2Si = irene.find_S2Si(S2, sipmzs)
            irene._store_pmaps(0, 0, S1, S2, S2Si)

            assert irene.s1t == pmap_file.root.PMAPS.S1
            assert irene.s2t == pmap_file.root.PMAPS.S2
            assert irene.s2sit == pmap_file.root.PMAPS.S2Si
            
    return CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si 



In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene, evt=0)

In [ ]:
wfm.plot_pmt_waveforms(CWF, zoom=False, window_size=2000)

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
tmin=590
tmax=620
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=100)

In [ ]:
tmin=0
tmax=590
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=5)

In [ ]:
S1

In [ ]:
S2

In [ ]:
pf.scan_s12(S2)

In [ ]:
pf.scan_s12(S1)

In [ ]:
s1par = S12P(tmin       = 800 * units.mus,
                 tmax   = 1100 * units.mus,
                 stride = 4,
                 lmin   = 6,
                 lmax   = 16,
                 rebin  = False)
irene2 = Irene(run_number  = 2980,
                 files_in    = [RWF_file],
                 file_out    = PMP_file,
                 compression = 'ZLIB4',
                 nprint      = 1,
                 n_baseline  = 38000,
                 thr_trigger = 5.0 * units.adc,
                 n_MAU       = 100,
                 thr_MAU     = 3.0 * units.adc,
                 thr_csum_s1 = 0.2 * units.adc,
                 thr_csum_s2 = 1.0 * units.adc,
                 n_MAU_sipm  = 100,
                 thr_sipm    = 5.0 * units.pes,
                 s1_params   = s1par,
                 s2_params   = s2par,
                 thr_sipm_s2 = 30. * units.pes)

In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene2, evt=0)

In [ ]:
S1

In [ ]:
pf.scan_s12(S1)

In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene2, evt=1)

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=120)

In [ ]:
S1

In [ ]:
S2

In [ ]:
pf.scan_s12(S2)

In [ ]:
pf.scan_s12(S1)

In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene2, evt=2)

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=120)

In [ ]:
S2

In [ ]:
S1

In [ ]:
pf.scan_s12(S1)

In [ ]:
wfm.plot_pmt_waveforms(CWF, zoom=False, window_size=2000)

In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene2, evt=3)

In [ ]:
wfm.plot_pmt_waveforms(CWF, zoom=False, window_size=2000)

In [ ]:
tmin=0
tmax=1200
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=150)

In [ ]:
pf.scan_s12(S2)

In [ ]:
pf.scan_s12(S1)

In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene, evt=3)

In [ ]:
pf.scan_s12(S1)

In [ ]:
CWF, csum, csum_mau, s1_ene, s2_ene, S1, S2, S2Si = irene_run(irene, evt=4)

In [ ]:
wfm.plot_pmt_waveforms(CWF, zoom=False, window_size=2000)

In [ ]:
tmin=490
tmax=625
mpl.plot_signal_vs_time_mus(csum, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=60)

In [ ]:
pf.scan_s12(S2)

In [ ]:
pf.scan_s12(S1)

In [ ]:
tmin=500
tmax=502
mpl.plot_signal_vs_time_mus(csum_mau, t_min=tmin, t_max=tmax, signal_min=-2, signal_max=6)